In [1]:
%load_ext autoreload
%autoreload 2

import time
import networkx as nx
import numpy as np
import logging
from copy import deepcopy
from collections import namedtuple
import torch
import pyro

from spatial_scene_grammars.nodes import *
from spatial_scene_grammars.rules import *
from spatial_scene_grammars.scene_grammar import *
from spatial_scene_grammars.parsing import *

In [2]:
from spatial_scene_grammars_examples.singles_pairs.grammar_constituency import *
#from spatial_scene_grammars_examples.singles_pairs.grammar_dependency import *
pyro.set_rng_seed(42)

grammar = SpatialSceneGrammar(
    root_node_type = Root,
    root_node_tf = torch.eye(4)
)
ground_truth_tree = grammar.sample_tree(detach=True)
observed_nodes = ground_truth_tree.get_observed_nodes()
print("Observed %d objects" % len(observed_nodes))
print("Objects: ", observed_nodes)

Observed 8 objects
Objects:  [<spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Root object at 0x7ff245005358>, <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Object object at 0x7ff245005f28>, <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Object object at 0x7ff245071898>, <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Object object at 0x7ff2450716d8>, <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Object object at 0x7ff245071a20>, <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Object object at 0x7ff245071a90>, <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Object object at 0x7ff245071b00>, <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Object object at 0x7ff245071b70>]


In [5]:
# Build up a big set of candidate intermediate nodes with both
# top-down and bottom-up generation.
candidate_intermediate_nodes = generate_candidate_intermediate_nodes(
    grammar, observed_nodes, max_recursion_depth=10, verbose=True
)

Candidate <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Singles object at 0x7ff2451f5f28> with pose tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])
Candidate <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Singles object at 0x7ff245097390> with pose tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])
Candidate <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Singles object at 0x7ff245097748> with pose tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])
Candidate <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Singles object at 0x7ff245097b70> with pose tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])
Candidate <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Singles object

In [6]:
parse_trees = infer_mle_tree_with_mip_from_proposals(
    grammar, observed_nodes, candidate_intermediate_nodes, verbose=True, N_solutions=1
)

Optimization success?:  True
Logfile: 

Gurobi 9.0.2 (linux64) logging started Tue Nov 30 18:15:46 2021

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 189 rows, 473 columns and 996 nonzeros
Model fingerprint: 0x8f935942
Variable types: 0 continuous, 473 integer (473 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 108 rows and 107 columns
Presolve time: 0.01s
Presolved: 81 rows, 366 columns, 838 nonzeros
Variable types: 0 continuous, 366 integer (366 binary)

Root relaxation: objective 7.691758e+02, 132 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                     769.1758337   -2.98656   100%     -    0s
     0     0     cutoff    0       769.17583  769.17583  0.00%     -  